In [1]:
import torch
import torch.nn as nn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator

from einops import rearrange, repeat, reduce

from darts.datasets import AirPassengersDataset

In [4]:
abc = torch.rand(5, 6, 2)
abc

tensor([[[0.4556, 0.0315],
         [0.1812, 0.9925],
         [0.9449, 0.0219],
         [0.1813, 0.3992],
         [0.3577, 0.4373],
         [0.4230, 0.7841]],

        [[0.0889, 0.5159],
         [0.5565, 0.2273],
         [0.2932, 0.3841],
         [0.2581, 0.4089],
         [0.9198, 0.3284],
         [0.6842, 0.0292]],

        [[0.4649, 0.4930],
         [0.9509, 0.1112],
         [0.0089, 0.8446],
         [0.7974, 0.9962],
         [0.6688, 0.1355],
         [0.6212, 0.1008]],

        [[0.3741, 0.4364],
         [0.7666, 0.7183],
         [0.9645, 0.3798],
         [0.8115, 0.5730],
         [0.0918, 0.5650],
         [0.1987, 0.0434]],

        [[0.5968, 0.5142],
         [0.8856, 0.1622],
         [0.1595, 0.9025],
         [0.7715, 0.4033],
         [0.9149, 0.0400],
         [0.8165, 0.4233]]])

In [7]:
input = abc.permute(0,2,1)
input

tensor([[[0.4556, 0.1812, 0.9449, 0.1813, 0.3577, 0.4230],
         [0.0315, 0.9925, 0.0219, 0.3992, 0.4373, 0.7841]],

        [[0.0889, 0.5565, 0.2932, 0.2581, 0.9198, 0.6842],
         [0.5159, 0.2273, 0.3841, 0.4089, 0.3284, 0.0292]],

        [[0.4649, 0.9509, 0.0089, 0.7974, 0.6688, 0.6212],
         [0.4930, 0.1112, 0.8446, 0.9962, 0.1355, 0.1008]],

        [[0.3741, 0.7666, 0.9645, 0.8115, 0.0918, 0.1987],
         [0.4364, 0.7183, 0.3798, 0.5730, 0.5650, 0.0434]],

        [[0.5968, 0.8856, 0.1595, 0.7715, 0.9149, 0.8165],
         [0.5142, 0.1622, 0.9025, 0.4033, 0.0400, 0.4233]]])

In [10]:
batch, channel, seq_len = input.shape
segments = seq_len // 3
input1 = input.view(batch, channel, segments, 3)
input1

tensor([[[[0.4556, 0.1812, 0.9449],
          [0.1813, 0.3577, 0.4230]],

         [[0.0315, 0.9925, 0.0219],
          [0.3992, 0.4373, 0.7841]]],


        [[[0.0889, 0.5565, 0.2932],
          [0.2581, 0.9198, 0.6842]],

         [[0.5159, 0.2273, 0.3841],
          [0.4089, 0.3284, 0.0292]]],


        [[[0.4649, 0.9509, 0.0089],
          [0.7974, 0.6688, 0.6212]],

         [[0.4930, 0.1112, 0.8446],
          [0.9962, 0.1355, 0.1008]]],


        [[[0.3741, 0.7666, 0.9645],
          [0.8115, 0.0918, 0.1987]],

         [[0.4364, 0.7183, 0.3798],
          [0.5730, 0.5650, 0.0434]]],


        [[[0.5968, 0.8856, 0.1595],
          [0.7715, 0.9149, 0.8165]],

         [[0.5142, 0.1622, 0.9025],
          [0.4033, 0.0400, 0.4233]]]])

In [11]:
channel_input1 = input1[:,1,:,:]
channel_input1

tensor([[[0.0315, 0.9925, 0.0219],
         [0.3992, 0.4373, 0.7841]],

        [[0.5159, 0.2273, 0.3841],
         [0.4089, 0.3284, 0.0292]],

        [[0.4930, 0.1112, 0.8446],
         [0.9962, 0.1355, 0.1008]],

        [[0.4364, 0.7183, 0.3798],
         [0.5730, 0.5650, 0.0434]],

        [[0.5142, 0.1622, 0.9025],
         [0.4033, 0.0400, 0.4233]]])

In [17]:
linear = nn.Linear(in_features=3, out_features=3, bias=False)
linear.weight = nn.Parameter(nn.init.orthogonal_(linear.weight))
out = linear(channel_input1)
out.shape

torch.Size([5, 2, 3])

In [18]:
w33 = torch.rand(10,1, 5,21)
print(w33.shape)
w33.squeeze(1).shape


torch.Size([10, 1, 5, 21])


torch.Size([10, 5, 21])